In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import re
import string
import pandas as pd

#Snorkel
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.model import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function

pd.set_option('display.max_colwidth', None)

### Loading and Preprocessing Textual Data From Twitter and News Headlines

#### Loading DataFrames

In [2]:
# Twitter
df_twitter = pd.read_pickle('../crawlers/twitter/all_tweets.pkl')
df_twitter.rename(columns={'text':'title'}, inplace=True)
df_twitter = df_twitter[['title']]
df_twitter['source'] = 'Twitter'
df_twitter.index = df_twitter.index.rename('date')
df_twitter.sample(5)

,title,source
date,,
2022-07-21 18:06:02,"petrobras: Insistência em nomes rejeitados para Petrobras é ataque à governança, dizem especialistas https://t.co/s3e7TWI8YL",Twitter
2022-07-18 08:58:48,WEG e Grupo Cevital fazem acordo para criar joint venture que produzirá motores elétricos - Finance News https://t.co/gd7Z46qj6l \n#Weg #WEGE3,Twitter
2022-07-20 13:57:31,Investidores dão dinheiro para um fundador de startup que já quebrou? https://t.co/ol1S20X4um,Twitter
2022-07-14 07:31:06,Cade investiga as maiores transportadoras de valores\nhttps://t.co/wtBhOX48IF,Twitter
2022-04-19 15:53:42,"""Ainda que outros riscos baixistas possam se concretizar, a invasão da Ucrânia pode gerar um potencial impacto de cerca de 1% sobre o IPCA do ano, tornando os riscos assimétricos para cima.""\n\nGauss Capital @gausscapital",Twitter


In [3]:
# News Headlines from Suno Research
df_news = pd.read_pickle('../crawlers/suno/all_news_suno.pkl')
df_news = df_news[['title']]
df_news['source'] = 'Suno'
df_news.sample(5)

,title,source
date,,
2021-04-14 14:59:00,Petrobras (PETR4) é impedida de distribuir dividendos após Justiça acatar pedido da Fup,Suno
2021-04-13 09:50:00,"Destaques de Empresas: Petrobras (PETR4), Banco do Brasil (BBAS3) e Neoenergia (NEOE3)",Suno
2021-08-21 18:37:00,"Petrobras (PETR4), Itaúsa (ITSA4): Veja as datas de corte e quem vai pagar dividendos na semana",Suno
2020-08-05 10:30:00,"Ibovespa abre em forte alta de 2,1%, a exemplo das bolsas mundiais",Suno
2020-06-08 11:47:00,"Banco do Brasil (BBAS3) adquire R$ 2,6 bi em carteiras do Banco Votorantim",Suno


In [4]:
# Append all dataframes

df_full = pd.DataFrame()

frames = [df_twitter, df_news]

for df in frames:
    df_full = df_full.append(df)

df_full.sample(5)

,title,source
date,,
2022-06-01 07:57:03,"Mercados nesta quarta, minério de ferro, petróleo e notícias de empresas da Bolsa - Finance News \nhttps://t.co/YsfNczSIzo\n#petróleo #minério #bolsas",Twitter
2022-06-17 18:05:28,"Petrobras não pode fechar as portas para governo federal, mas obrigações têm que ser mantidas, diz Marcelo Gasparino\nhttps://t.co/C4JA2X4hHD",Twitter
2022-06-25 00:08:52,Quina de São João sorteia prêmio de R$ 200 milhões hoje e não acumula\nhttps://t.co/p5Wgf9YyYX,Twitter
2016-05-04 09:35:15,"A gasolina Podium mantém o sistema de combustão, os bicos injetores e as válvulas do motor limpos, reduzindo o custo de manutenção.",Twitter
2022-06-18 15:08:54,Quanto vale um almoço com o megainvestidor Warren Buffett?\nhttps://t.co/a22BHUm9SD,Twitter


In [5]:
df_full.source.value_counts()

Twitter    39157
Suno        2650
Name: source, dtype: int64

#### Preprocessing Data

In [6]:
def remove_emojis(sentence):

    "Remoção de Emojis nas mensagens de texto."

    # Padrões dos Emojis
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', sentence)

def remove_valores(sentence):
    new_sentece = ''
    
    for token in sentence.split():
        if token.isdigit():
            token = '<NUM>'
        new_sentece += ' {}'.format(token)
        
    return new_sentece

def remove_links(sentence):
    new_sentece = ''
    
    for token in sentence.split():
        if token.startswith('http'):
            token = ''
        new_sentece += ' {}'.format(token)
        
    return new_sentece

In [7]:
def apply_preprocessing(df_input: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica preprocessamento nos títulos e textos completos
    """
    
    df = df_input.copy()

    # Substituir símbolos importantes
    df['title'] = df['title'].map(lambda s: s.replace('-feira', ''))
    df['title'] = df['title'].map(lambda s: s.replace('-alvo', ' alvo'))
    df['title'] = df['title'].map(lambda s: s.replace('\n', ' '))  
    df['title'] = df['title'].map(lambda s: s.replace('+', ''))
    df['title'] = df['title'].map(lambda s: s.replace('º', ''))
    df['title'] = df['title'].map(lambda s: s.replace("‘", ''))
    df['title'] = df['title'].map(lambda s: s.replace("’", ''))
    df['title'] = df['title'].map(lambda s: s.replace("•", ''))
    df['title'] = df['title'].map(lambda s: s.replace('-', ''))
    df['title'] = df['title'].map(lambda s: s.replace('%', ' por cento'))
    df['title'] = df['title'].map(lambda s: s.replace('R$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('U$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('US$', ''))
    df['title'] = df['title'].map(lambda s: s.replace('S&P 500', 'spx'))
    df['title'] = df['title'].map(lambda s: s.replace('/', '@'))

    # Remove Links, Hashtags e Menções
    df['title'] = df['title'].map(lambda s: remove_links(s))
    df['title'] = df['title'].str.replace('(\#\w+.*?)',"")
    df['title'] = df['title'].str.replace('(\@\w+.*?)',"")

    # Transformar em String e Letras Minúsculas nas Mensagens
    df['title'] = df['title'].map(lambda s: str(s).lower())

    # Remover Pontuações
    df['title'] = df['title'].map(lambda s: s.translate(str.maketrans('', '', string.punctuation)))

    # Remover Emojis     
    df['title'] = df['title'].map(lambda s: remove_emojis(s))

    # Quebras de Linha desnecessárias
    df['title'] = df['title'].map(lambda s: s.replace('\n', ' '))

    # Remover aspas duplas
    df['title'] = df['title'].map(lambda s: s.replace('\"', ''))
    df['title'] = df['title'].map(lambda s: s.replace('“', ''))
    df['title'] = df['title'].map(lambda s: s.replace('”', ''))

    # Remover valores
    df['title'] = df['title'].map(lambda s: remove_valores(s))

    # Espaços desnecessários
    df['title'] = df['title'].map(lambda s: s.strip())

    return df

In [8]:
df_full_preprocessed = apply_preprocessing(df_input=df_full)
df_full_preprocessed.sample(10)

,title,source
date,,
2016-03-31 16:01:05,procurando por postos petrobras que vendem o diesel s10 encontre na busca de postos,Twitter
2022-06-03 21:00:59,especial livros execução – a disciplina para atingir resultados,Twitter
2022-07-11 10:01:50,perspectivas do mercado para a semana os próximos dias prometem ser agitados em termos de informações com potencial para mexer com a dinâmica,Twitter
2022-07-22 18:51:02,tese de investimentos eficiência e expansão da prio animam analistas,Twitter
2022-06-24 17:19:30,tempo para abrir nova empresa cai para menos de três dias pela primeira vez aponta sebrae,Twitter
2021-06-18 16:07:00,concessão do metrôrio será assumida pelo mubadala que toma o lugar da invepar,Suno
2021-01-12 08:22:29,ministro busca avanço de reformas para empresas e empregos a informação foi fornecida nesta segunda <NUM> pelo ministério da economia,Twitter
2022-06-30 23:07:21,ipea eleva previsão de alta do pib de <NUM> para <NUM> por cento mas reduz a de <NUM> a <NUM> por cento,Twitter
2022-03-06 16:10:01,não consegue se organizar para quitar aquelas antigas confira dicas anote suas fixas mensais e total que você deve pague <NUM> as dívidas menores tente renegociar ou parcelar as dívidas maiores no cartão de crédito evite pagar só o valor mínimo,Twitter


In [10]:
df_full.loc['2022-06-24 17:19:30']

,title,source
date,,
2022-06-24 17:19:30,"Tempo para abrir nova empresa cai para menos de três dias pela primeira vez, aponta Sebrae\nhttps://t.co/6ntsgKc1X2",Twitter


---

#### Create Train/Test Splits

### Creating Labelling Functions (LFs)

In [11]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

categories = [POSITIVE, NEGATIVE, ABSTAIN]

#### Positive Sentiment LFs

- Adjectives
- Verbs

In [15]:
@labeling_function()
def lf_news_good_adjs(x):
    with open('./dicts/final/pos_adj.txt') as file:
        adjectives = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in adjectives:
            return POSITIVE
    return ABSTAIN

@labeling_function()
def lf_happiness_words(x):
    with open('./dicts/dicts_emocoes/alegria.txt') as f_words_happiness:
        hapiness_words = [line.rstrip() for line in f_words_happiness]
    
    for word in x.title.lower().split():
        if word in hapiness_words:
            return POSITIVE
    return ABSTAIN

@labeling_function()
def lf_news_good_verbs(x):
    with open('./dicts/pos_verbs.txt') as file:
        verbs = [line.rstrip() for line in file]
    
    for word in x.title.lower().split():
        if word in verbs:
            return POSITIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_dividendos(x):
    dividend_pattern = r".*pag.*dividendo.*|.*anunc.*dividendo.*|.*distrib.*dividendo.*"
    return POSITIVE if re.search(dividend_pattern, x.title.lower(), flags=re.I) else ABSTAIN

@labeling_function()
def lf_regex_resultado_positivo(x):
    raise_pattern = r"fech.*alta.*|.*abr.*alta.*|.*fech.*pos.*|.*abr.*pos.*|.*estre.*alta.*|.*prev.*alta.*|.*result.*positivo.*" 
    return POSITIVE if re.search(raise_pattern, x.title.lower(), flags=re.I) else ABSTAIN

#### Negative Sentiment LFs

In [16]:
@labeling_function()
def lf_news_bad_adjs(x):
    with open('./dicts/final/neg_adj.txt') as f_adj_neg:
        adjectives = [line.rstrip() for line in f_adj_neg]

    for word in x.title.lower().split():
        if word in adjectives:
            return NEGATIVE

    return ABSTAIN

@labeling_function()
def lf_sadness_words(x):
    with open('./dicts/dicts_emocoes/tristeza.txt') as f_words_sadness:
        sadness_words = [line.rstrip() for line in f_words_sadness]
    
    for word in x.title.lower().split():
        if word in sadness_words:
            return NEGATIVE

    return ABSTAIN

@labeling_function()
def lf_news_bad_verbs(x):
    with open('./dicts/neg_verbs.txt') as f_verb_neg:
        verbs = [line.rstrip() for line in f_verb_neg]
    
    for word in x.title.lower().split():
        if word in verbs:
            return NEGATIVE
        
    return ABSTAIN

@labeling_function()
def lf_regex_resultado_negativo(x):
    fall_pattern = r"fech.*queda.*|.*abr.*queda.*|.*fech.*neg.*|.*abr.*neg.*|.*prev.*baixa.*|.*prev.*queda.*|.*em.*queda.*|.*result.*negativo.*" 
    return NEGATIVE if re.search(fall_pattern, x.title.lower(), flags=re.I) else ABSTAIN

In [17]:
# combine all the labeling functions 

df = df_full_preprocessed.copy()

df = df[['title']]

lfs = [
       lf_news_good_adjs,
       lf_happiness_words,
       lf_news_good_verbs,
       lf_regex_dividendos,
       lf_regex_resultado_positivo,
       lf_news_bad_adjs,
       lf_sadness_words,
       lf_news_bad_verbs,
       lf_regex_resultado_negativo
]


# apply the label model
applier = PandasLFApplier(lfs=lfs)

label_model = LabelModel(cardinality=len(categories),
                         device='cpu', 
                         verbose=False)

# apply the lfs on the dataframe
L_train = applier.apply(df=df, 
                        progress_bar=False)

# fit on the data
label_model.fit(L_train,
                n_epochs=1500,
                log_freq=100, 
                seed=123)

# predict and create the labels
df['label'] = label_model.predict(L=L_train, 
                                  tie_break_policy='abstain').astype(str)

# Convert Labels to Real classes
dict_map = {'-1': 'NEUTRAL', '1': 'POSITIVE', '0': 'NEGATIVE'}
df['label_class'] = df['label'].map(dict_map)

100%|██████████| 1500/1500 [00:00<00:00, 1590.77epoch/s]


In [18]:
# Polarity: The set of unique labels this LF outputs (excluding abstains)
# Coverage: The fraction of the dataset that each LF labels
# Overlaps: The fraction of the dataset where this LF and at least one other LF label
# Conflicts: The fraction of the dataset where this LF and at least one other LF label and disagree

LFAnalysis(L=L_train, lfs=lfs).lf_summary()#[['Coverage', 'Overlaps', 'Conflicts']] * 100

,j,Polarity,Coverage,Overlaps,Conflicts
lf_news_good_adjs,0,[1],0.126821,0.054417,0.017820
lf_happiness_words,1,[1],0.114287,0.050805,0.014280
lf_news_good_verbs,2,[1],0.043366,0.013634,0.003660
lf_regex_dividendos,3,[1],0.011410,0.003779,0.000359
lf_regex_resultado_positivo,4,[1],0.013227,0.005095,0.002703
lf_news_bad_adjs,5,[0],0.044945,0.020044,0.015572
lf_sadness_words,6,[0],0.025091,0.013084,0.007989
lf_news_bad_verbs,7,[0],0.016337,0.006913,0.004856
lf_regex_resultado_negativo,8,[0],0.016983,0.006937,0.006123


In [19]:
df[['title', 'label_class']].sample(10)

,title,label_class
date,,
2017-05-31 14:00:36,copom hoje na tlivltgogt acompanhe em tempo real notícias e insights que movem o mercado,POSITIVE
2022-06-21 18:10:48,preço dos novos contratos de aluguel sobe <NUM> por cento em <NUM> meses em sp,NEUTRAL
2021-12-08 22:00:56,petrorio prio3 – produção de novembro soma <NUM> boepd queda de <NUM> por cento ante outubro,NEGATIVE
2022-07-21 13:50:49,bolsas da europa fecham sem direção única após decisão do bce e instabilidade na itália,NEUTRAL
2022-05-12 10:27:58,a principal tendência é bearish,NEUTRAL
2021-12-10 16:00:01,coluna no aniversário da declaração universal dos direitos humanos não há nada o que comemorar no país,POSITIVE
2022-06-08 08:33:26,índices futuros de wall street operam em baixa à espera de novos dados de inflação,NEUTRAL
2022-06-02 04:05:10,mercado vivo e oi ameaçam devolver concessões da telefonia fixa para a união,NEUTRAL
2022-06-30 09:34:44,executivo de valor premia os melhores gestores do país,POSITIVE


In [20]:
# Filtering out unlabeled data points
df = df.loc[df.label_class.isin(['POSITIVE', 'NEGATIVE']), :]

print ('Quantidade Total de Mensagens Rotuladas: ', df.shape[0])

# Find label counts 
df['label_class'].value_counts() / df.shape[0] * 100

Quantidade Total de Mensagens Rotuladas:  12846


POSITIVE    78.818309
NEGATIVE    21.181691
Name: label_class, dtype: float64

In [21]:
df.sample(10)

,title,label,label_class
date,,,
2022-07-20 20:59:49,tesla vendeu <NUM> milhões em bitcoin no segundo trimestre,0,NEGATIVE
2022-03-09 22:20:00,radar via viia3 tem queda no lucro do 4t21 amazon amzo34 aprova desdobramento de ações cade mantém venda da oi oibr3,1,POSITIVE
2022-06-22 12:50:30,shiba inu a subida parece ter acabado – cair abaixo de <NUM>,0,NEGATIVE
2022-06-27 12:18:07,flerta com <NUM> mil pontos apoiado em e,1,POSITIVE
2022-07-20 11:40:14,comitiva brasileira vai aos eua alertar sobre ameaças às eleições <NUM>,1,POSITIVE
2022-07-13 18:19:07,dividendos de <NUM> mil reais por mês com fiis conheça a estratégia da influenciadora carol dias para investir,1,POSITIVE
2022-03-22 20:16:03,weg poderá adquirir até <NUM> mil ações ordinárias finance news,1,POSITIVE
2021-12-22 09:32:41,listinha para começar o dia bem critérios para começar investindo certo na antes de tudo descubra seu perfil de e onde é mais indicado defina onde você quer chegar seus objetivos saiba avaliar as das empresas disponíveis anotou,1,POSITIVE
2022-06-14 14:58:37,tesouro superávit primário de abril é o maior para o mês desde <NUM>,1,POSITIVE


---